<a href="https://colab.research.google.com/github/Andreavisi1/Challenge_B1_PHM_North_America_2023/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/ProgettoManutenzionePreventiva/Data_Challenge_PHM2023_test_data.zip" "/tmp/"

!unzip -oq "/tmp/Data_Challenge_PHM2023_test_data.zip" -d "/tmp"

!cp "/content/drive/MyDrive/ProgettoManutenzionePreventiva/Data_Challenge_PHM2023_training_data.zip" "/tmp/"

!unzip -oq "/tmp/Data_Challenge_PHM2023_training_data.zip" -d "/tmp"

In [ ]:
# Converts relative paths to absolute ones
ROOT_TRAIN_DATA_FOLDER ='/tmp/Data_Challenge_PHM2023_training_data'
ROOT_VAL_DATA_FOLDER = '/tmp/Data_Challenge_PHM2023_test_data'

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re

In [ ]:
def parse_vibration_dataset(dataset_path):
    """
    Parsa il dataset di vibrazione e crea un DataFrame pandas con una riga per ogni misura.

    Args:
        dataset_path (str): Percorso alla cartella principale del dataset

    Returns:
        pd.DataFrame: DataFrame con colonne etichetta, velocità, torque, rep e dati di vibrazione (una riga per campione)
    """

    data_list = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.txt'):
                folder_name = os.path.basename(root)
                if 'Pitting_degradation_level_' in folder_name:
                    etichetta_full = folder_name.replace('Pitting_degradation_level_', '')
                    if '(' in etichetta_full:
                        etichetta = etichetta_full.split('(')[0].strip()
                        descrizione = etichetta_full.split('(')[1].replace(')', '').strip()
                    else:
                        etichetta = etichetta_full.strip()
                        descrizione = None
                else:
                    etichetta = folder_name
                    descrizione = None

                pattern = r'V(\d+)_(\d+)N_(\d+)\.txt'
                match = re.search(pattern, file)

                if match:
                    velocita = int(match.group(1))
                    torque = int(match.group(2))
                    rep = int(match.group(3))

                    file_path = os.path.join(root, file)
                    try:
                        data = np.loadtxt(file_path)

                        for row in data:
                            record = {
                                'etichetta': etichetta,
                                'velocita': velocita,
                                'torque': torque,
                                'rep': rep,
                                'horizontal_acceleration': row[0],
                                'axial_acceleration': row[1],
                                'vertical_acceleration': row[2],
                                'tachometer_signal': row[3],
                            }
                            data_list.append(record)

                    except Exception as e:
                        print(f"Errore nel leggere il file {file_path}: {e}")
                        continue
                else:
                    print(f"Nome file non riconosciuto: {file}")

    df = pd.DataFrame(data_list)

    if not df.empty:
        df = df.sort_values(['etichetta', 'velocita', 'torque', 'rep']).reset_index(drop=True)

    return df

def get_vibration_statistics(df):
    """
    Calcola statistiche sui dati di vibrazione, raggruppando per file.

    Args:
        df (pd.DataFrame): DataFrame con una riga per ogni misura

    Returns:
        pd.DataFrame: DataFrame con statistiche per ogni combinazione (etichetta, velocità, torque, rep)
    """

    stats_list = []

    grouped = df.groupby(['etichetta', 'descrizione', 'velocita', 'torque', 'rep'])

    for (etichetta, descrizione, velocita, torque, rep), group in grouped:
        h_acc = group['horizontal_acceleration'].values
        a_acc = group['axial_acceleration'].values
        v_acc = group['vertical_acceleration'].values

        stats = {
            'etichetta': etichetta,
            'descrizione': descrizione,
            'velocita': velocita,
            'torque': torque,
            'rep': rep,
            # Statistiche accelerazione orizzontale
            'h_acc_mean': np.mean(h_acc),
            'h_acc_std': np.std(h_acc),
            'h_acc_rms': np.sqrt(np.mean(h_acc ** 2)),
            'h_acc_max': np.max(np.abs(h_acc)),
            # Statistiche accelerazione assiale
            'a_acc_mean': np.mean(a_acc),
            'a_acc_std': np.std(a_acc),
            'a_acc_rms': np.sqrt(np.mean(a_acc ** 2)),
            'a_acc_max': np.max(np.abs(a_acc)),
            # Statistiche accelerazione verticale
            'v_acc_mean': np.mean(v_acc),
            'v_acc_std': np.std(v_acc),
            'v_acc_rms': np.sqrt(np.mean(v_acc ** 2)),
            'v_acc_max': np.max(np.abs(v_acc)),
            # Info generali
            'num_samples': len(group)
        }

        stats_list.append(stats)

    return pd.DataFrame(stats_list)



In [ ]:
df_vibration = parse_vibration_dataset(ROOT_TRAIN_DATA_FOLDER)

if not df_vibration.empty:
    print(f"Dataset caricato con successo!")
    print(f"Numero di file processati: {len(df_vibration)}")
    print(f"Etichette uniche: {df_vibration['etichetta'].unique()}")
    print(f"Descrizioni uniche: {df_vibration['descrizione'].unique()}")
    print(f"Velocità uniche: {sorted(df_vibration['velocita'].unique())}")
    print(f"Torque unici: {sorted(df_vibration['torque'].unique())}")
    print(f"Repetition uniche: {sorted(df_vibration['rep'].unique())}")

    # Mostra le prime righe (senza i dati di vibrazione per leggibilità)
    print("\nPrime righe del dataset:")
    display_cols = ['etichetta', 'descrizione', 'velocita', 'torque', 'rep', 'num_samples', 'file_path']
    print(df_vibration[display_cols].head())

    # Calcola statistiche
    print("\nCalcolo delle statistiche...")
    df_stats = get_vibration_statistics(df_vibration)
    print("\nStatistiche per i primi file:")
    print(df_stats.head())

    # Esempio di accesso ai dati di vibrazione per un file specifico
    print(f"\nEsempio - forma dei dati del primo file:")
    print(f"Accelerazione orizzontale: {df_vibration.iloc[0]['horizontal_acceleration'].shape}")
    print(f"Primi 5 valori accelerazione orizzontale: {df_vibration.iloc[0]['horizontal_acceleration'][:5]}")

else:
    print("Nessun file trovato o errore nel parsing del dataset")

In [ ]:
df_vibration.head()